# Notes to self:
Next time start by: 

Other things: 



# General Setup

In [1]:
import os
#Specify whether to use CHEESEHEAD or synthtic data 
os.environ['config'] = 'CHEESEHEAD'
# os.environ['config'] = 'synthetic'

from setup import * #Import setup module

from chad_funcs import *


# #If using limited cases- full case list read in through setup module
# cases = pd.read_csv('../Inputs/cases_limited.csv', index_col = 'case') #dataframe of tower coordinates

In [2]:
# # Reimport
# import importlib
# import setup
# importlib.reload(setup)
# from setup import*

# Functions

## Nan Counts

In [3]:
def nancounts(data, varlist, levlist):
    nancount =make_multi_df([varlist, levlist], ['var', 'lev'], dtindex) #Integer
    nanmask = make_multi_df([varlist, levlist, towlist], ['var', 'lev', 'tow'], dtindex)#Boolean

    for var in varlist: #Loop through both variable
        for lev in levlist: #Loop through all three levels
            nanmask[var, lev] = data[var][lev].notnull();
            nancount[var, lev] = 18 - data[var][lev].count(axis=1)
            
    return nanmask, nancount

## Corrdinates

### NSEW

In [ ]:
#Function to calculate coordinates at different dirs NSEW from each tower
def NSEWcoords(dist, tc = tc): 
    tc_int = tc.copy() #Make copy of tc to put interpolation coords in
    coords_dist = {} #Dictionary of NSEW coords at the specified distance
    
    #N, S, E, W coords (in km)
    tc_int['y_N'] = tc_int['y'] + dist
    tc_int['y_S'] = tc_int['y'] - dist
    tc_int['x_E'] = tc_int['x'] + dist
    tc_int['x_W'] = tc_int['x'] - dist

    #Dictionary holding 2D array of coords for each tower in each direction 
    #Dims are tower, time, and direction (x/y)
    coords_dist['N'] = np.stack((tc_int.x.values, tc_int.y_N.values)).T
    coords_dist['S'] = np.stack((tc_int.x.values, tc_int.y_S.values)).T
    coords_dist['E'] = np.stack((tc_int.x_E.values, tc_int.y.values)).T
    coords_dist['W'] = np.stack((tc_int.x_W.values, tc_int.y.values)).T
    
    return coords_dist

### Up/down

In [5]:
def updowncoords(dist, WD, tc = tc):
    
    coords_dist = pd.DataFrame(index = dtindex, columns = towu2dirdim_cols)
    WD = WD.astype(float)
    #Locations dist km up and down wind from 
    coords_dist.loc[:, (slice(None), 'up', 'x')] = tc.x.values + dist*np.sin(WD*np.pi/180).values
    coords_dist.loc[:, (slice(None), 'up', 'y')] = tc.y.values + dist*np.cos(WD*np.pi/180).values
    coords_dist.loc[:, (slice(None), 'down', 'x')] = tc.x.values - dist*np.sin(WD*np.pi/180).values
    coords_dist.loc[:, (slice(None), 'down', 'y')] = tc.y.values - dist*np.cos(WD*np.pi/180).values
    
    #Create dictionary 
    
    return coords_dist

### Geometry - convert NSEW to updown

In [6]:
def grad_NSEW_to_updown(hrz_wind, grad_NSEW):
    WD_rad = hrz_wind['WD']*np.pi/180 #Convert to radians
    NS_grad = grad_NSEW.xs("NS", level="direct", axis=1, drop_level=True) #grad_NSEW.loc[:, (slice(None), 'NS')]
    EW_grad = grad_NSEW.xs("EW", level="direct", axis=1, drop_level=True)
    #Note this is the same as grad_NSEW.loc[:, (slice(None), 'EW')].droplevel(leve = 'direct', axis = 1)
    grad_updown = NS_grad*np.cos(WD_rad.astype(float)) + EW_grad*np.sin(WD_rad.astype(float))
    return grad_updown

## Interpolation 

### NSEW

In [7]:
def interp_NSEW(data, coords_dist, dtindex, nancount_varlev, nanmask_varlev, kernel = 'thin_plate_spline'):
    
    #Create interp_dat dataframe with nested columns (tower>direction)
    interped_dat = make_multi_df([towlist, NSEWdirlist], ["tower", "direct"], index = dtindex)

    if kernel == 'quintic':
        nancount_cutoff = 13
    else:
        nancount_cutoff = 16
    
    #Loop through timestep and interpolate
    for i in dtindex:
        dat_use = data.loc[data.index == i].values[0, :] #For each loop interpolate with just one row of data

        dat_nn = dat_use[nanmask_varlev.loc[i]] #dat_use without nan values #fisthis - get rid of varlev
        ar_nn = ar[nanmask_varlev.loc[i]] #coordinates of towers that have finite values
        
        if nancount_varlev.loc[i] >= nancount_cutoff:
            pass
            
        else:
            #Run interpolator
            interpfunction = RBFInterpolator(ar_nn, dat_nn, kernel = kernel, epsilon = 1)

            #Apply interpolation function to coords of points in each direction
            for direct in NSEWdirlist:
                interped_dat.loc[i, idx[:, direct]] = interpfunction(coords_dist[direct])
                
     
            
    return interped_dat

### Up/down

In [8]:
def interp_updown(data, coords_dist, dtindex, nancount_varlev, nanmask_varlev, kernel = 'thin_plate_spline'):

    
    #Create interp_dat dataframe with nested columns (tower>direction)
    interped_dat = make_multi_df([towlist, u2_dirlist], ["tower", "direct"], index = dtindex)
    
    if kernel == 'quintic':
        nancount_cutoff = 13
    else:
        nancount_cutoff = 16
    
    #Loop through timestep and interpolate
    for i in dtindex:
        dat_use = data.loc[data.index == i].values[0, :] #For each loop interpolate with just one row of data
        
        dat_nn = dat_use[nanmask_varlev.loc[i]] #dat_use without nan values
        ar_nn = ar[nanmask_varlev.loc[i]] #coordinates of towers that have finite values
        
        if nancount_varlev.loc[i] >= nancount_cutoff:
            pass
            
        else:

            #Run interpolator
            interpfunction = RBFInterpolator(ar_nn, dat_nn, kernel = kernel, epsilon = 1)

            #Appy interp function to coords in each direction
            for direct in u2_dirlist:
                coords_use = np.stack((coords_dist.loc[i, (slice(None), direct, 'x')].values,
                                       coords_dist.loc[i, (slice(None), direct, 'y')].values)).T
                interped_dat.loc[i, idx[:, direct]] = interpfunction(coords_use)
            
    return interped_dat

## Gradients

### NSEW

In [9]:
#Gradient calcs

def NSEWgradcalc(hrz_wind, interpdat, dtindex, nancount, nanmask, varlist, levlist,
                 interpdist = 1000, kernel = 'thin_plate_spline'):
    
    #Calc coords of interp locations
    coords_dist = NSEWcoords(interpdist)
    
    #Create output grad df
    grad_updown = make_multi_df([varlist, levlist, towlist], ['var', 'lev', 'tow'], dtindex)    
    
    
    for var in varlist: #Loop through variable
        for lev in levlist: #Loop through all three levels
            data_use = interpdat[var][lev]
            pointdat = interp_NSEW(data_use, coords_dist, dtindex, nancount[var][lev], nanmask[var][lev], 
                                   kernel) #gradients for particular var, lev, 
            grad_NSEW = NSEW_point_to_grad(pointdat, dtindex, interpdist)
            grad_updown[var, lev] = grad_NSEW_to_updown(hrz_wind, grad_NSEW)
           
    
    return grad_updown


#Converts point data to gradients
def NSEW_point_to_grad(pointdat, dtindex, dist):
    grad = pd.DataFrame(index = dtindex, columns = towc2dir_cols)
    grad.loc[:, (slice(None), 'NS')] = \
    (pointdat.loc[:, (slice(None), 'N')].values - pointdat.loc[:, (slice(None), 'S')].values)/(2*dist)
    grad.loc[:, (slice(None), 'EW')] = \
    (pointdat.loc[:, (slice(None), 'E')].values - pointdat.loc[:, (slice(None), 'W')].values)/(2*dist)

    return grad


### Up/down

In [10]:
#Updown calcs

def updowngradcalc(hrz_wind, interpdat, dtindex, nancount, nanmask, varlist, levlist,
                   interpdist = 1000, kernel = 'thin_plate_spline'):
    
    #Coordinates for points up and downwind of towers
    coords_dist = updowncoords(interpdist, hrz_wind['WD'].astype(float))
    
    #Set up df to receive grad data
    grad_updown = make_multi_df([varlist, levlist, towlist], ['var', 'lev', 'tow'], dtindex)
    
    for var in varlist: #Loop through variable
        for lev in levlist: #Loop through all three levels
            data_use = interpdat[var][lev]
            pointdat = interp_updown(data_use, coords_dist, dtindex, nancount[var][lev], nanmask[var][lev],
                                     kernel) #gradients for particular var, lev, all tow, dirs
            grad_updown[var, lev] = updown_point_to_grad(pointdat, dtindex, interpdist)
    
    
    return grad_updown 



#Converts point data to gradients #fixthis1
def updown_point_to_grad(pointdat, dtindex, dist):
    grad_dat = (pointdat.loc[:, (slice(None), 'up')].values - pointdat.loc[:, (slice(None), 'down')].values)/(2*dist)
    grad = pd.DataFrame(index = dtindex, columns = towlist, data = grad_dat)
    return grad


### Weighted Gradient Averaging

In [11]:
def gradient_averaging(hrz_wind, interpdat, dtindex, tc, varlist, levlist):
    #Calculated gradients for each var, lev, tow, NS/EW
    grad_updown = make_multi_df([varlist, levlist, towlist], ['var', 'lev', 'tow'], dtindex)
    grad_calc = make_multi_df([varlist, levlist, towlist, c2_dirlist], ['var', 'lev', 'tow', 'direct'], dtindex)
    #Distance between each pair of towers

    #Loop through each tower
    for tow1 in towlist: #Tower that we want to calc gradietnts for
        #Create data frame to calc gradients between each pair of towers, to be averaged   

        for var in varlist: #Loop through variable
            for lev in levlist: #Loop through all three levels
                data_use = interpdat[var][lev]
                gradx, grady = one_tow_grad_avg(tow1, data_use, tc)
                grad_updown[var, lev, tow1] = one_tow_grad_NSEW_to_updown(hrz_wind, tow1, gradx, grady)
    return grad_updown

#### one_tow_grad_avg

In [12]:
def one_tow_grad_avg(tow1, data_use, tc):

    sum_weights = np.zeros(len(dtindex))
    gradx_sum = np.zeros(len(dtindex))
    grady_sum = np.zeros(len(dtindex))
    

    for tow2 in towlist:
        hrz_dists = pd.DataFrame(columns = towlist)
        if tow1 == tow2: #No need to compare to same tower
            pass
        else:
            deltax = tc.loc[tow1].x - tc.loc[tow2].x #x distance between two towers
            deltay = tc.loc[tow1].y - tc.loc[tow2].y #y distance between two towers
            hrz_dists = (deltax**2 + deltay**2)**(1/2) #Total horizontal distance between towers
            weight = 1/hrz_dists 

            delta_var = data_use[tow1] - data_use[tow2] #Difference in var from tow1 to tow2
            grad_var = delta_var/(np.abs(deltax) + np.abs(deltay)) #assuming same grad in x and y dirs
            gradx = np.nan_to_num(grad_var*deltax/np.abs(deltax)*weight) #need sign of deltax and y
            grady = np.nan_to_num(grad_var*deltay/np.abs(deltay)*weight) #need sign of deltax and y
            gradx_sum = gradx_sum + gradx
            grady_sum = grady_sum + grady

            
            #Timeseries with weight where gradient is finite, 0 if gradient is nan
            time_weights = np.where(np.isfinite(grad_var) == True, weight, 0)
            sum_weights = sum_weights + time_weights #Add up weights for each tower

    sum_weights = np.where(sum_weights > 0, sum_weights, np.nan) #If sum_weights is 0, set to nan
    gradx_mean = gradx_sum/sum_weights
    grady_mean = grady_sum/sum_weights
    
    
    
    return gradx_mean, grady_mean
    

    
        

#### one_tow_grad_NSEW_to_updown

In [13]:
#Function for NSEW gradients to updown, one tower at a time, just for gradient averaging method    
def one_tow_grad_NSEW_to_updown(hrz_wind, tow, EW_grad, NS_grad):
    WD_rad = hrz_wind['WD'][tow]*np.pi/180 #Convert to radians
    grad_updown = NS_grad*np.cos(WD_rad.astype(float)) + EW_grad*np.sin(WD_rad.astype(float))
    return grad_updown    

## Wind Dat Calc

In [14]:
#This only changes for stability method, otherwise is same for all cases
#Currently calculating wind speed at 33 m for all towers regardless of z1. Could just use WS_top for z1>30

def U_33_calc(stab_meth, MO, hrz_wind):
    WS_33 = pd.DataFrame(index = dtindex, columns = towlist)
    
    
    for tow in towlist:
        
        #Tower constants
        h_veg = tc.veg_h.loc[tow] #vegetation height
        z_Umes= tc.z1.loc[tow] #Wind measurement height
        z0 = 0.1*h_veg #friction parameter
        z0_soil = 0.006 #From biophysic notes, check book Table_, ch.5
        d = tc.d.loc[tow] #zero plane displacement
        a = 1 #From Campbell and Norman table 5.2 for Larch trees
        
        Psi_Umes = stab_cor(stab_meth, z_Umes, z0, MO[tow], dtindex)
        ustar_calc = hrz_wind['WS_top'][tow]*k/np.log((z_Umes - d)/z0 + Psi_Umes) #ustar to use in wind profile
        u_of_h = ustar_calc/k*np.log((h_veg-d)/z0) #Wind speed at top of canopy
        u_of_soil = u_of_h*np.exp(a*(z0/h_veg - 1)) #WS at 0.1h, using exponential sub-canopy model
        ustar_soil = u_of_soil*k/np.log(z0/z0_soil) #Soil friction velocity, calculated from WS at 01.h
        

        z = 33 #Get wind speed at 33 m for all towers

        WS_33[tow] = WS_of_z(stab_meth, z, z0, d, h_veg, z0_soil, u_of_h, ustar_calc, ustar_soil, MO[tow], dtindex)
        
        
        #Dataframe to hold u and v
    U_33 = make_multi_df([U_varlist, U_levlist, towlist], ['var', 'lev', 'tow'], dtindex)

    #Calculate u and v from WD - currently at top for each tower
    WD_rad = hrz_wind['WD']*np.pi/180 #Convert to radians
    U_33['u', 33] = WS_33*-np.sin(WD_rad.astype(float))
    U_33['v', 33] = WS_33*-np.cos(WD_rad.astype(float))

    stab_meth_old = stab_meth
    return U_33

# Setup for this notebook

## Read in data

In [15]:
#Read in data 
startdate = dt.datetime(2019, 6, 20)
enddate = dt.datetime(2019, 10, 13, 23, 30)
# startdate = dt.datetime(2019, 7, 1)
# enddate = dt.datetime(2019, 7, 2)

dtindex = readdata('dtindex', startdate, enddate) #Save dtindex as separate variable
hrz_wind = readdata('hrz_wind', startdate, enddate) #Create function to set this if I try different hrz_wind calcs #fixthis
tracdat = {'TA': readdata('TA', startdate, enddate),
              'H2O': readdata('H2O', startdate, enddate)} #data to be inerpolated
MO = readdata('MO', startdate, enddate)


## Interpolation setup

In [16]:
#Make a 2D array with x and y values for location of each tower used for interpolation
ar = np.array([tc.x.values, tc.y.values]).T

#nancount and mask for tracer data


In [17]:
##Creat nanmask and nancount arrays
nanmask, nancount = nancounts(tracdat, trac_varlist, trac_levlist)
nanmask.to_pickle(intermed_filepath + 'nandat/trac_nanmask.pickle')
nancount.to_pickle(intermed_filepath + 'nandat/trac_nancount.pickle')

# Run interpolation - T and q

In [ ]:
# kernel = 'thin_plate_spline'
step = 0.1 #Step for vertical integration
save = True
interpdat = tracdat
varlist, levlist = trac_varlist, trac_levlist



for case in cases.index:
    #Case parameters
    grad_method = cases['grad_method'].loc[case]
    interpdist = cases['interp_dist'].loc[case]
    interpdir = cases['interp_dir'].loc[case]
    stab_meth = cases['stability'].loc[case]
    kernel = cases['interp_kernel'].loc[case]
    
        
    ##Creat nanmask and nancount arrays
    nanmask, nancount = nancounts(interpdat, varlist, levlist)
    
    #Run interpolation
    if grad_method == 'gradavg':
        grad_updown = gradient_averaging(hrz_wind, interpdat, dtindex, tc, varlist, levlist)
        filepath = intermed_filepath + 'hrz_gradients/'+ grad_method + '.pickle'
        
    elif grad_method == 'interp':
        if interpdir == 'NSEW':
            grad_updown = NSEWgradcalc(hrz_wind, interpdat, dtindex, nancount, nanmask, 
                                       varlist, levlist, interpdist, kernel)
        elif interpdir == 'updown':
            grad_updown = updowngradcalc(hrz_wind, interpdat, dtindex, nancount, nanmask, 
                                         varlist, levlist, interpdist, kernel)
        filepath = intermed_filepath + 'hrz_gradients/tracer' + grad_method + str(interpdist) + interpdir + kernel + '.pickle'
    
    # # Filepath for shortened time
    filepath = intermed_filepath + 'hrz_gradients/tracer' + grad_method + str(interpdist) + str(interpdir) \
     + str(kernel) +str(stab_meth) + startdate.strftime("%m%d")+ '-'+ enddate.strftime("%m%d")+'.pickle'
    
    if save == True:
        with open(filepath, 'wb') as handle:
            pickle.dump(grad_updown, handle)
        

# Calc Vertical Velocity

## Wind Interpolation

### Setup and get wind data

In [20]:
stab_meth = None#'Benoit'
interpdist = 1000
kernel = 'thin_plate_spline'
step = 0.1
z_mes = 33 #Height of interpolated wind data


#calc U_33
U_33 = U_33_calc(stab_meth, MO, hrz_wind)

#Calc coords of interp locations
coords_dist = NSEWcoords(interpdist)

##Creat nanmask and nancount arrays
nanmask, nancount = nancounts(U_33, U_varlist, U_levlist)

In [21]:
#Cal U at 33 m at NSEW points
pointdat = make_multi_df([U_varlist, U_levlist, towlist, NSEWdirlist], 
                          ['var', 'lev', 'tow', 'direct'], dtindex)    


for var in U_varlist: #Loop through variable
    for lev in U_levlist: #Only on level, matching format for tracer interp
        data_use = U_33[var][lev]
        pointdat[var, lev] = interp_NSEW(data_use, coords_dist, dtindex, nancount[var][lev], nanmask[var][lev], 
                               kernel) #gradients for particular var, lev, 

In [23]:
w_cont = pd.DataFrame(columns = towlist, index = dtindex)

for tow in towlist:
    u_E = wind_prof(tow, stab_meth, pointdat.loc[:, ('u', 33, tow, 'E')], MO[tow], dtindex,
                    step, z_mes)
    u_W = wind_prof(tow, stab_meth, pointdat.loc[:, ('u', 33, tow, 'W')], MO[tow], dtindex,
                    step, z_mes)
    v_N = wind_prof(tow, stab_meth, pointdat.loc[:, ('v', 33, tow, 'N')], MO[tow], dtindex,
                    step, z_mes)
    v_S = wind_prof(tow, stab_meth, pointdat.loc[:, ('v', 33, tow, 'S')], MO[tow], dtindex,
                    step, z_mes)
    
    dudx = (u_E - u_W)/(2*interpdist) #positive sign is divergence, neg convergence
    dvdy = (v_N - v_S)/(2*interpdist) #positive sign is divergence, neg convergence
    dwdz = -dudx - dvdy
    w_cont[tow] = dwdz.sum(axis = 1)
    
# save w_cont to inputs
w_cont.to_pickle(input_filepath + 'w_cont_lai.pickle')    

In [ ]:
#Move all this into different notebook later

In [ ]:
w_son = readdata('w_son', startdate, enddate) 

In [ ]:
d = 5
m = 9

cut_startdate = dt.datetime(2019, m, d )
cut_enddate = dt.datetime(2019, m, d + 5)
w_cont_cut = w_cont.loc[cut_startdate:cut_enddate]
w_son_cut = w_son.loc[cut_startdate:cut_enddate]

In [ ]:
w_son_nonan = w_son.dropna(subset=['PFb', 'PFc']).astype('float')

slope, intercept, r_value, p_value, std_err = stats.linregress(w_son_nonan['PFb'], w_son_nonan['PFc'])

In [ ]:
r_value

In [ ]:
x = np.arange(-0.5, 0.8, 0.1)
y = x*slope + intercept


plt.scatter(w_son_nonan['PFb'], w_son_nonan['PFc'], s = 1)

plt.plot(x, y, color = "xkcd:terracotta", linewidth = 1.5)

# Get current axes
ax = plt.gca()

# # Set the position of the spines to cross at (0,0)
# ax.spines['left'].set_position('zero')
# ax.spines['bottom'].set_position('zero')

# # Hide the top and right spines
# ax.spines['right'].set_color('none')
# ax.spines['top'].set_color('none')

ax.grid(True, which='both')

# ax.set_xlabel('nw1 w [m/s]', labelpad= -30, ha='right', x=0.97)
# ax.set_ylabel('nw2 w [m/s]', labelpad=-30, va='top', y=1.05, rotation = 0 )

ax.set_xlabel('nw1 w [m/s]')
ax.set_ylabel('nw2 w [m/s]')
plt.tight_layout()

# plt.xlabel('w nw1 [m/s]')
# plt.ylabel('w nw2 [m/s]')

In [ ]:
ax = plt.gca()
ax.axhline(y=0, color='gray', linestyle='--', linewidth=1)
for tow in towlist[1:5]:
    plt.plot(w_son_cut[tow], label = tc.loc[tow].wind_var[-3:])
plt.xticks(rotation = 45);
plt.legend();
plt.ylabel('w [m/s]')

In [ ]:
tc.loc['PFb']['org_path']

In [ ]:
ax = plt.gca()
ax.axhline(y=0, color='gray', linestyle='--', linewidth=1)

plt.plot(w_son_cut['PFb'], label = 'nw1 w sonic')
plt.plot(w_cont_cut['PFb'], label = 'nw1 w continuity')
plt.plot(w_son_cut['PFc'], label = 'nw2 w sonic')
plt.plot(w_cont_cut['PFc'], label = 'nw2 w continuity')
plt.xticks(rotation = 45);
plt.legend();
plt.ylabel('w [m/s]')

# ax = plt.gca()
# ax.axhline(y=0, color='gray', linestyle='--', linewidth=1)

In [ ]:
plt.scatter(w_son['PFb'], w_son['PFc'])

In [ ]:
tow = 'PFb'
fig, ax1 = plt.subplots()
plt.plot(w_cont_cut[tow], label = 'continuity');
plt.plot(-w_son_cut[tow], label = 'sonic');
# ax2 = ax1.twinx()
# ax2.plot(WD_cut[tow], label = 'wind dir', color = 'k')
fig.legend()
plt.title(tow)
plt.ylabel('w_cont, w_son')
# plt.ylabel('w [m/s]')
plt.xticks(rotation = 45);


In [ ]:
h = 7
lag_list = np.arange(-6, 6)
corr = pd.DataFrame(index = towlist, columns = lag_list)
for lag in lag_list:
    cut1_startdate = dt.datetime(2019, 6, 21, h)
    cut1_enddate = dt.datetime(2019, 10, 13, h)
    cut2_startdate = dt.datetime(2019, 6, 21, h+lag)
    cut2_enddate = dt.datetime(2019, 10, 13, h+lag)
    w_cont_cut = w_cont.loc[cut1_startdate:cut1_enddate]
    w_son_cut = w_son.loc[cut2_startdate:cut2_enddate]
    for tow in towlist[1:]:
        x, y = w_son_cut[tow].values.astype('float'), w_cont_cut[tow].values.astype('float')
        nas = np.logical_or(np.isnan(x), np.isnan(y))
        corr.loc[tow, lag] = scipy.stats.pearsonr(x[~nas], y[~nas])[0]
    

In [ ]:
corr = pd.Series(index = towlist)
for tow in towlist[1:]:
    x, y = w_son_cut[tow].values.astype('float'), w_cont_cut[tow].values.astype('float')
    nas = np.logical_or(np.isnan(x), np.isnan(y))
    corr.loc[tow] = scipy.stats.pearsonr(x[~nas], y[~nas])[0]

In [ ]:
#color list
colorlist = ["xkcd:ocean blue", "xkcd:terracotta", "xkcd:blue green","xkcd:grape",  "xkcd:grey",  
             "xkcd:forrest green","xkcd:magenta", "xkcd:grey blue", "xkcd:dark yellow","xkcd:coral",
              "xkcd:sky blue", "xkcd:light orange","xkcd:reddish brown", "xkcd:dark lavender",  "xkcd:light olive",
             "xkcd:puke yellow", "xkcd:light cyan", "xkcd:orchid",  "xkcd:periwinkle blue"]
mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=colorlist) 

In [ ]:
scatter = plt.scatter(tc.x, tc.y, c=corr, cmap='PuOr')
plt.colorbar(scatter, label='Value')
plt.xlabel('x [m]')
plt.ylabel('y [m]')
plt.show()

# Archive

In [ ]:
# tow = 'PFA'
# WS_use_tow = pointdat.loc[:, ('u', 33, tow, 'E')]
# #Tower constants
# step = 0.1
# h_veg = tc.veg_h.loc[tow] #vegetation height
# z_son = round(tc.z_son.loc[tow], 1) #tower height, rounded to nearest 0.1, don't actually need this now but in case of use with more precise heights
# z_range = np.arange(0, z_son + step, step) #need wind profile up to level of sonic
# z_Umes = 33
# z0 = 0.1*h_veg #friction parameter
# z0_soil = 0.006 #From biophysic notes, check book Table_, ch.5
# d = tc.d.loc[tow] #zero plane displacement
# a = a_default #Attenuation coeficient

# WS_prof = pd.DataFrame(index = dtindex, columns = z_range) #windspeed profile

# Psi_z1 = stab_cor(stab_meth, z_Umes, z0, MO[tow], dtindex)
# ustar_calc = WS_use_tow*k/np.log((z_Umes - d)/z0 + Psi_z1) #ustar to use in wind profile
# u_of_h = ustar_calc/k*np.log((h_veg-d)/z0) #Wind speed at top of canopy
# u_of_soil = u_of_h*np.exp(a*(z0/h_veg - 1)) #WS at 0.1h, using exponential sub-canopy model
# ustar_soil = u_of_soil*k/np.log(z0/z0_soil) #Soil friction velocity, calculated from WS at 01.h
    
    
# WS_of_z(stab_meth, h, z0, d, h_veg, z0_soil, u_of_h, ustar_calc, ustar_soil, MO[tow], dtindex)

In [ ]:
# timestamp = dtindex[0]
# TA_2 = interpdat['TA'].loc[2, timestamp]
# plt.tricontourf(tc.x, tc.y, TA_2)

In [ ]:
#Check updown coords visually

# WD = readdata('hrz_wind', startdate, enddate).loc['WD']
# ud_coords = updowncoords(0.5, WD)

# timestep = 320 #Angle (1 degree +1 per timestamp)

# up_x = ud_coords.loc[ud_coords.index[timestep], (slice(None), 'up', 'x')].values
# up_y = ud_coords.loc[ud_coords.index[timestep], (slice(None), 'up', 'y')].values

# down_x = ud_coords.loc[ud_coords.index[timestep], (slice(None), 'down', 'x')].values
# down_y = ud_coords.loc[ud_coords.index[timestep], (slice(None), 'down', 'y')].values

# plt.scatter(tc.x, tc.y)
# plt.scatter(up_x, up_y)
# plt.scatter(down_x, down_y)

In [ ]:
# #Create dtindex file in case I mess it up again

# dtindex = hrz_wind.loc['WD'].index

# #Filepath for full time
# filepath = '../Inputs/dtindex.pickle'

# with open(filepath, 'wb') as handle:
#     pickle.dump(dtindex, handle)

In [ ]:
## Save all interp coords (rather than calculating separately for each run)
# coords = {}
# coords['NSEW'] = {}
# coords['updown'] = {}

# for dist in distlist: 
#     coords['NSEW'][dist] = NSEWcoords(dist) #Nested dictionary - [distance][direction] - 2 cols (x and y coords)
#     coords['updown'][dist] = updowncoords(dist, unidata, tc)

In [ ]:
# #Interpolate to up and downwind points, rather than N/S, E/W points
# x_up =  pd.DataFrame(index=df_TA.index)
# x_down =  pd.DataFrame(index=df_TA.index)
# y_up =  pd.DataFrame(index=df_TA.index)
# y_down =  pd.DataFrame(index=df_TA.index)
# for tower in tc.index:
# #     x_coord[str(tower + '_up')] = tc['x'][tower] + np.cos(PFA_interp.WD_1_1_1*np.pi/180)
# #     x_coord[str(tower + '_down')] = tc['x'][tower] - np.cos(PFA_interp.WD_1_1_1*np.pi/180)
    
# #     y_coord[str(tower + '_up')] = tc['y'][tower] + np.sin(PFA_interp.WD_1_1_1*np.pi/180)
# #     y_coord[str(tower + '_down')] = tc['y'][tower] - np.sin(PFA_interp.WD_1_1_1*np.pi/180)
    
    
#     x_up[tower] = tc['x'][tower] + np.cos(PFA_interp.WD_1_1_1*np.pi/180)
#     x_down[tower] = tc['x'][tower] - np.cos(PFA_interp.WD_1_1_1*np.pi/180)
    
#     y_up[tower] = tc['y'][tower] + np.sin(PFA_interp.WD_1_1_1*np.pi/180)
#     y_down[tower] = tc['y'][tower] - np.sin(PFA_interp.WD_1_1_1*np.pi/180)
    
    
# #Interpolate to up and downwind coords (comment this section!)

# #Make a 2D array with x and y values for location of each tower
# ar = np.array([tc.y.values, tc.x.values]).T

# #3D arrays of locations 1km up and down wind of each tower 
# #Dims are tower, time, and direction (x/y)
# up_coords = np.stack((np.asarray(x_up), np.asarray(y_up))).T
# down_coords = np.stack((np.asarray(x_down), np.asarray(y_down))).T


# #Initiate lists to recieve up and downwind temperatures
# T_i_up = []
# T_i_down = []

# #Loop through each time stamp and interpolate to up and downwind coordinates
# #Is there a way to do this without a for loop - not sure since the coords change with each time step
# #It doesn't take too long as is so maybe fine this way
# #Could use points N/S, E/W of towers, use x and y components of velocity
# for i in range(len(df_TA)):
#     T_i_up.append(RBFInterpolator(ar, df_TA.values.T[:, i])(up_coords[:, i, :]))
#     T_i_down.append(RBFInterpolator(ar, df_TA.values.T[:, i])(down_coords[:, i, :]))

    
# #Convert lists to arrays
# T_i_up = np.asarray(T_i_up)
# T_i_down = np.asarray(T_i_down)


In [ ]:
# # Plot profiles 

# #profile_plot(30_m_data, 10_m_data, 2_m_data, folder_for_plot_storage, x_axis_range)

# def profile_plot(data_1, data_2, data_3, folder, x_range = [-4, 4]):
#     data_mon_1 = data_1.groupby([data_1.index.month, data_1.index.time]).mean()
#     data_mon_2 = data_2.groupby([data_2.index.month, data_2.index.time]).mean()
#     data_mon_3 = data_3.groupby([data_3.index.month, data_3.index.time]).mean()
    
    
    
#     for tower in tc.index:
        
#         if towdat.z3 == 0:
#             pass
        
#         else:

#             fig, axs = plt.subplots(12, 4, figsize = (10, 30))
#             for mon in range(6, 11):
#                 i=0
#                 for hour in data_mon_1.index.get_level_values(1).drop_duplicates():
#                     ax = axs[i//4, i%4]
#                     i= i+1
                    
#                     dif_top = data_mon_1.loc[(mon, hour), tower] - data_mon_2.loc[(mon, hour), tower]
#                     dif_bottom = data_mon_3.loc[(mon, hour), tower] - data_mon_2.loc[(mon, hour), tower]
#                     # dif_bottom = data_mon[mon][hour][tower] - data_mon[mon][hour]tower]

#                     # ax.plot([df_mon.TA_1_1_1[mon][hour], df_mon.TA_1_2_1[mon][hour],df_mon.TA_1_3_1[mon][hour]], [towdat.z1, towdat.z2, towdat.z3])
#                     ax.plot([dif_top, 0, dif_bottom], [towdat.z1, towdat.z2, towdat.z3])
#                     ax.set_yticks([towdat.z3, towdat.z2, towdat.z1])
#                     ax.set_title(hour)
#                     ax.set_xlim(x_range)

#             fig.suptitle(tower)
#             plt.tight_layout()
#             plt.savefig(str('./Outputs/diff_Profiles/Interp_dirs/' + folder + '/' +tower + '_TA_diff_profiles'))


In [ ]:
# #Calc horizontal distances between each pair of towers
# hrz_dists = pd.DataFrame(index = towlist, columns = towlist)
# #For each tow1, loop through other towers to calc gradients between them
# for tow2 in towlist: #All other towers to calc gradients from
#     if tow1 == tow2: #No need to compare to same tower
#         pass
#     else:
#         deltax = tc.loc[tow1].x - tc.loc[tow2].x #x distance between two towers
#         deltay = tc.loc[tow1].y - tc.loc[tow2].y #y distance between two towers
#         hrz_dists.loc[tow2, tow1] = (deltax**2 + deltay**2)**(1/2) #Total horizontal distance between towers